# Laboratorio #5 – Threat hunting

# Librerías

In [229]:
import json
import pandas as pd
import tldextract

# Parte 1 – Filtrado y preprocesamiento

## 1. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total

In [230]:
dataFromJson = []
fileName = "large_eve.json"
with open(fileName, "r", encoding="utf-8") as file:
    for line in file:
        dataFromJson.append(json.loads(line))

print("Data len = ", len(dataFromJson))

Data len =  746909


## 2. Debido a que estamos buscando dominios web, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS.

In [231]:
dnsRecords = []

for record in dataFromJson:
    if record["event_type"] == "dns":
        dnsRecords.append(record)

## 3. Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad mucho más manejable, pero aún se debe seguir depurando la información a buscar

In [232]:
print("Number of DNS records: ", len(dnsRecords))

Number of DNS records:  15749


## 4. Muestre la información de 2 registros cualesquiera

In [233]:
print("First DNS record: ", dnsRecords[0])
print("Last DNS record: ", dnsRecords[1])

First DNS record:  {'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
Last DNS record:  {'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}


## 5. Debido a que la data consiste en estructuras JSON anidadas, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe. Muestre el shape del dataframe, debería obtener (21484, 163)

In [234]:
dataframe = pd.json_normalize(dnsRecords)
dataframe.shape

(15749, 18)

In [235]:
dataframe.head(1)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN


## 6. Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio). Después de filtrar debería obtener 2849 registros

In [236]:
dataframe = dataframe[dataframe["dns.rrtype"] == "A"]
dataframe.shape

(2849, 18)

## 7. Filtre los dominios únicos. Debe obtener 177 registros únicos

In [237]:
dataframe.drop_duplicates(subset=["dns.rrname"], inplace=True)

In [238]:
dataframe.shape

(177, 18)

## 8. Escriba una función que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home. Utilice un LLM para ayudarle a escribir esta función, verifique que obtiene correctamente el TLD, incluya el prompt utilizado en su notebook.

Prompt:
<pre>
hola. tengo un dataframe el cual tiene la siguiente estructura:
<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, 0 to 15737
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   177 non-null    object 
 1   flow_id     177 non-null    int64  
 2   pcap_cnt    177 non-null    int64  
 3   event_type  177 non-null    object 
 4   vlan        177 non-null    int64  
 5   src_ip      177 non-null    object 
 6   src_port    177 non-null    int64  
 7   dest_ip     177 non-null    object 
 8   dest_port   177 non-null    int64  
 9   proto       177 non-null    object 
 10  dns.type    177 non-null    object 
 11  dns.id      177 non-null    int64  
 12  dns.rrname  177 non-null    object 
 13  dns.rrtype  177 non-null    object 
 14  dns.tx_id   177 non-null    float64
 15  dns.rcode   0 non-null      object 
 16  dns.ttl     0 non-null      float64
 17  dns.rdata   0 non-null      object 
dtypes: float64(2), int64(6), object(10)
memory usage: 26.3+ KB


donde la columna "dns.rrname" contiene el dominio. 
aqui te dejo unos posibles valores de la columna en cuestión:
0                              api.wunderground.com
1                               stork79.dropbox.com
2        hpca-tier2.office.aol.com.ad.aol.aoltw.net
6              safebrowsing.clients.google.com.home
9                               fxfeeds.mozilla.com
                            ...                    
15632                    ntp.ubuntu.com.localdomain
15661              redir.metaservices.microsoft.com
15701                             ocsp.verisign.com
15716                192.168.22.201:.stayonline.net
15737                      client-software.real.com
Name: dns.rrname, Length: 177, dtype: object


necesito que me ayudes a crear una función en python capaz de obtener el TLD para un cierto dominio. 
Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para <br>safebrowsing.clients.google.com.home, el TLD es home
</pre>

Respuesta del LLM:

Para crear una función en Python que obtenga el TLD (Top-Level Domain) de un dominio, podemos utilizar la biblioteca `tldextract`, que es muy útil para extraer los componentes de un dominio. A continuación, te muestro cómo puedes hacerlo:

1. Primero, instala la biblioteca `tldextract` si no la tienes instalada:
    ```sh
    pip install tldextract
    ```

2. Luego, puedes definir una función que utilice `tldextract` para obtener el TLD de un dominio. Aquí tienes un ejemplo de cómo hacerlo:



In [239]:
import tldextract

def obtener_tld(dominio):
    ext = tldextract.extract(dominio)
    if ext.suffix:
        return f"{ext.domain}.{ext.suffix}"
    else:
        return ext.domain

# Ejemplos de uso
dominios = [
    "api.wunderground.com",
    "stork79.dropbox.com",
    "hpca-tier2.office.aol.com.ad.aol.aoltw.net",
    "safebrowsing.clients.google.com.home",
    "fxfeeds.mozilla.com"
]

for dominio in dominios:
    print(f"Dominio: {dominio}, TLD: {obtener_tld(dominio)}")

Dominio: api.wunderground.com, TLD: wunderground.com
Dominio: stork79.dropbox.com, TLD: dropbox.com
Dominio: hpca-tier2.office.aol.com.ad.aol.aoltw.net, TLD: aoltw.net
Dominio: safebrowsing.clients.google.com.home, TLD: home
Dominio: fxfeeds.mozilla.com, TLD: mozilla.com




Este código definirá una función `obtener_tld` que toma un dominio como entrada y devuelve el TLD correspondiente. Luego, se muestran algunos ejemplos de uso de la función.

Puedes incluir este código en tu notebook y verificar que obtienes correctamente el TLD para los dominios proporcionados.

## 9. Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando la función anterior para crear una columna nueva llamada domain_tld, y elimine todas las demás columnas.

In [240]:
dataframe["domain_tld"] = dataframe["dns.rrname"].apply(obtener_tld)

In [241]:
dataframe.head(1)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,domain_tld
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com


In [242]:
dataframe = dataframe["domain_tld"].reset_index(inplace=False).drop(columns=["index"])

In [ ]:
dataframe.dropna(subset=["domain_tld"], inplace=True)

In [243]:
dataframe.head(20)

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com
5,metasploit.com
6,aol.com
7,aoltw.net
8,aol.com
9,aol.com


# Parte 2 – Data Science

In [247]:
geminiDataframe = dataframe.copy()

## 10. Utilice Gemini para clasificar los dominios como DGA (1) o legítimos (0)

In [250]:
geminiDataframe.head()

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com


In [ ]:
import google.generativeai as genai
from google.api_core import retry
import enum

class DomainType(enum.Enum):
    MALICIOUS = "malicious"
    BENIGN = "benign"

genai.configure(api_key="aqui-iría-la-llave.SI-TUVIERA-UNA")
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}
model = genai.GenerativeModel('gemini-1.5-flash-001',
                                generation_config=genai.GenerationConfig(
                                    response_mime_type="text/x.enum",
                                    response_schema=DomainType
                                ))

prompt = """Clasifica el dominio como MALICIOUS o BENIGN.
Dominio: {domain}"""

In [279]:
# Voy a guardar el geminiDataframe en un archivo CSV para futuras ejecuciones ya que seguro me voy a acabar mi cuota de consumo y necesitaré hacerlo varios días. Pero antes voy a agregar una columna primero con valor None y así solamente envíar a Gemini los dominios que no han sido procesados
geminiDataframe["dga"] = None
geminiDataframe.to_csv("geminiDataframe.csv", index=False)

In [346]:
geminiDataframe = pd.read_csv("geminiDataframe.csv")
geminiDataframe["dga"] = geminiDataframe["dga"].astype("Int64")
geminiDataframe.replace({pd.NA: None}, inplace=True)

In [347]:
geminiDataframe.head()

,domain_tld,dga
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,1
3,home,0
4,mozilla.com,0


In [353]:
BATCH_SIZE = 10
for i in range(0, len(geminiDataframe), BATCH_SIZE):
    print(f"Haciendo el batch de {i} a {i+BATCH_SIZE}")
    batch = geminiDataframe[i:i+BATCH_SIZE]
    toProcess = batch[batch["dga"].isnull()]
    if toProcess.empty:
        print("Los dominios ya han sido procesados")
        continue
    for index, row in toProcess.iterrows():
        domain = row["domain_tld"]
        if row["dga"] is pd.NA:
            continue
        try:
            response = model.generate_content(prompt.format(domain=domain), request_options=retry_policy)
            if response is not None and len(response.parts) != 0:
                response = response.text
            else:
                response = "malicious"
            response = 1 if response == "malicious" else 0
            print(f"Dominio: {domain}, Tipo: {response}")
            geminiDataframe.loc[index, "dga"] = response
        except Exception as e:
            print(f"Error en dominio {domain}: {e}")
    if input("Continuar? ") != "s":
        break

Haciendo el batch de 0 a 10
Los dominios ya han sido procesados
Haciendo el batch de 10 a 20
Los dominios ya han sido procesados
Haciendo el batch de 20 a 30
Los dominios ya han sido procesados
Haciendo el batch de 30 a 40
Dominio: freepbx.org, Tipo: 0
Dominio: bigflickrfeed.com, Tipo: 1
Dominio: gnu.org, Tipo: 0
Dominio: wpad, Tipo: 1
Dominio: lan, Tipo: 0
Dominio: 56", Tipo: 0
Dominio: acunetix.com, Tipo: 0
Dominio: windowsupdate.com, Tipo: 0
Dominio: flickr.com, Tipo: 0
Dominio: malwarecity.com, Tipo: 1
Haciendo el batch de 40 a 50
Dominio: facebook.com, Tipo: 0
Dominio: securityfocus.com, Tipo: 0
Dominio: xmarks.com, Tipo: 0
Dominio: 0, Tipo: 1
Dominio: cakephp.org, Tipo: 0
Dominio: hec.net, Tipo: 0
Dominio: adams.net, Tipo: 0
Dominio: uidaho.edu, Tipo: 0
Dominio: pdx.edu, Tipo: 0
Dominio: clarkson.edu, Tipo: 0
Haciendo el batch de 50 a 60
Dominio: rackspace.com, Tipo: 0
Dominio: ecvps.com, Tipo: 1
Dominio: wisc.edu, Tipo: 0
Dominio: facebook.net, Tipo: 0
Dominio: easynews.com, Tip

In [354]:
geminiDataframe.to_csv("geminiDataframe.csv", index=False)
geminiDataframe.head(100)

,domain_tld,dga
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,1
3,home,0
4,mozilla.com,0
...,...,...
95,macromates.com,0
96,254,0
97,ejfodfmfxlkgifuf.xyz,1
98,192.168.22.201,0


## 11. Filtre aquellos considerados como DGA (valor 1 ) y muéstrelos. Recuerde que los resultados de los modelos pueden incluir falsos positivos y falsos negativos, por lo que no podemos fiarnos por completo de esta clasificación y debemos seguir indagando. Después de eliminar duplicados, debe obtener 61 registros únicos.

In [364]:
geminiDataframe[geminiDataframe["dga"] == 0]

,domain_tld,dga
0,wunderground.com,0
1,dropbox.com,0
3,home,0
4,mozilla.com,0
6,aol.com,0
...,...,...
95,macromates.com,0
96,254,0
98,192.168.22.201,0
99,verisign.com,0


# Parte 3 – Dominio experto

## 12. Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo que debemos aplicar dominio experto para encontrar los verdaderos registros maliciosos. Escriba una función que utilice la lista de un millón de TLD proporcionada en Canvas, y devuelva 0 si el TLD se encuentra en la lista y 1 si no está

Prompt:
<pre>
hola. necesito tu ayuda para crear una función la cuál primero lea un csv de nombre "top-1m.csv" 
el cual contiene una lista de un millón de TLD. donde el csv empieza de esta forma: 
"1,google.com 2,www.google.com 3,microsoft.com 4,data.microsoft.com 5,events.data.microsoft.com 
6,live.com 7,office.com 8,apple.com 9,microsoftonline.com 10,login.microsoftonline.com" 
luego recorrer un dataframe en una variable de nombre 'dataframe' de una sola columna de nombre 
"domain_tld" y verificar si existe dentro de la lista de un millón o no, devolver 0 si el TLD se 
encuentra en la lista y 1 si no está.  
ten cuidado en no leer el csv cada vez que se busca un TLD para evitar cargas innecesarias. 
agregar a la variable 'dataframe' la columna correspodiente a si está o no en la lista de un millón 
de TLD
</pre>
Respuesta del LLM:

Para crear la función que necesitas, primero leeremos el archivo CSV "top-1m.csv" una sola vez y almacenaremos los TLD en un conjunto para una búsqueda rápida. Luego, recorreremos el DataFrame `dataframe` y verificaremos si cada TLD está en la lista de un millón de TLDs. Finalmente, agregaremos una nueva columna al DataFrame con el resultado de la verificación.

Aquí tienes el código para lograr esto:



In [224]:
import pandas as pd

# Leer el archivo CSV y almacenar los TLD en un conjunto
def cargar_tlds(filepath):
    df_tlds = pd.read_csv(filepath, header=None, usecols=[1], names=['tld'])
    tlds_set = df_tlds['tld']
    return tlds_set.tolist()

# Función para verificar si el TLD está en la lista de un millón de TLDs
def verificar_tld(dataframe, tlds_set):
    dataframe['en_lista'] = dataframe['domain_tld'].apply(lambda x: 0 if x in tlds_set else 1)
    return dataframe

# Ruta al archivo CSV
filepath = 'top-1m.csv'

# Cargar los TLDs en un conjunto
tlds_set = cargar_tlds(filepath)

# Ejemplo de DataFrame
dataframePrueba = pd.DataFrame({
    'domain_tld': [
        'google.com',
        'unknown-domain.com',
        'microsoft.com',
        'example.com'
    ]
})

# Verificar los TLDs y agregar la columna correspondiente
dataframePrueba = verificar_tld(dataframePrueba, tlds_set)

# Mostrar el DataFrame resultante
print(dataframePrueba)

           domain_tld  en_lista
0          google.com         0
1  unknown-domain.com         1
2       microsoft.com         0
3         example.com         0




Este código realiza las siguientes acciones:

1. Define la función `cargar_tlds` para leer el archivo CSV "top-1m.csv" y almacenar los TLDs en un conjunto.
2. Define la función `verificar_tld` para recorrer el DataFrame `dataframe` y verificar si cada TLD está en el conjunto de TLDs cargados.
3. Carga los TLDs desde el archivo CSV utilizando `cargar_tlds`.
4. Verifica los TLDs en el DataFrame `dataframe` y agrega una nueva columna `en_lista` con el resultado de la verificación.

Puedes ajustar el código según sea necesario para tu entorno y datos específicos.

## 13. Utilice la función para determinar si los TLD se encuentran en dicha lista. Filtre aquellos que si se encuentran. Después de eliminar duplicados, debería obtener 13 dominios

In [225]:
dataframe = verificar_tld(dataframe, tlds_set)

In [227]:
dataframe.drop_duplicates(subset=["domain_tld"], inplace=True)

In [196]:
dataframe[dataframe["en_lista"] == 1].shape

(42, 2)

In [197]:
dataframe[dataframe["en_lista"] == 1].head(100)

,domain_tld,en_lista
2,aoltw.net,1
3,home,1
14,stayonline.net,1
18,110phpmyadmin,1
19,informaction.com,1
20,localdomain,1
27,ntkrnlpa.info,1
33,arrancar.org,1
34,sql-ledger.org,1
35,backtrack-linux.org,1


## 14. Finalmente, para confirmar los dominios maliciosos podemos buscar la fecha de creación del TLD. Cree una función qué en base al TLD, devuelva la fecha de creación de este. 

Prompt: 
<pre>
basado en el dataframe resultante quiero que implementes otra función. necesito poder obtener 
la fecha de creación del TLD, por lo que necesito crear una función capaz de obtener la 
fecha de creación de un TLD
</pre>
Respuesta del LLM:

Para obtener la fecha de creación de un TLD, podemos utilizar la biblioteca `whois` en Python, que permite consultar información WHOIS sobre dominios. A continuación, te muestro cómo puedes crear una función para obtener la fecha de creación de un TLD y aplicarla al DataFrame resultante.

Primero, instala la biblioteca `whois` si no la tienes instalada:


In [ ]:
pip install python-whois



Luego, puedes definir la función para obtener la fecha de creación del TLD y aplicarla al DataFrame:



In [ ]:
import whois
import pandas as pd

# Función para obtener la fecha de creación de un TLD
def obtener_fecha_creacion(tld):
    try:
        dominio_info = whois.whois(tld)
        return dominio_info.creation_date
    except Exception as e:
        return None

# Ejemplo de DataFrame resultante
dataframePrueba = pd.DataFrame({
    'domain_tld': [
        'google.com',
        'unknown-domain.com',
        'microsoft.com',
        'example.com'
    ],
    'en_lista': [0, 1, 0, 1]
})

# Aplicar la función al DataFrame para obtener la fecha de creación
dataframePrueba['fecha_creacion'] = dataframePrueba['domain_tld'].apply(obtener_fecha_creacion)

# Mostrar el DataFrame resultante
print(dataframePrueba)

           domain_tld  en_lista  \
0          google.com         0   
1  unknown-domain.com         1   
2       microsoft.com         0   
3         example.com         1   

                                     fecha_creacion  
0  [1997-09-15 04:00:00, 1997-09-15 07:00:00+00:00]  
1                               2020-04-11 19:59:53  
2  [1991-05-02 04:00:00, 1991-05-02 04:00:00+00:00]  
3                               1995-08-14 04:00:00  




Este código realiza las siguientes acciones:

1. Define la función `obtener_fecha_creacion` que utiliza la biblioteca `whois` para obtener la fecha de creación de un TLD.
2. Aplica la función `obtener_fecha_creacion` a la columna `domain_tld` del DataFrame resultante y agrega una nueva columna `fecha_creacion` con la fecha de creación del TLD.

Puedes ajustar el código según sea necesario para tu entorno y datos específicos. Ten en cuenta que la consulta WHOIS puede no funcionar para todos los dominios y puede haber limitaciones en la cantidad de consultas que puedes realizar en un período de tiempo.

In [199]:
dataframe['fecha_creacion'] = dataframe['domain_tld'].apply(obtener_fecha_creacion)

In [205]:
dataframe.head()

,domain_tld,en_lista,fecha_creacion
0,wunderground.com,0,"[1995-03-30 05:00:00, 1995-03-30 00:00:00]"
1,dropbox.com,0,"[1995-06-28 04:00:00, 1995-06-28 04:00:00+00:00]"
2,aoltw.net,1,2000-01-10 04:09:19
3,home,1,None
4,mozilla.com,0,1994-10-18 04:00:00


## 15. Muestre la fecha de creación para cada uno de los 13 dominios finales

In [206]:
dataframe[dataframe["en_lista"] == 1]

,domain_tld,en_lista,fecha_creacion
2,aoltw.net,1,2000-01-10 04:09:19
3,home,1,None
14,stayonline.net,1,1999-12-14 19:57:16
18,110phpmyadmin,1,None
19,informaction.com,1,1998-11-25 05:00:00
20,localdomain,1,None
27,ntkrnlpa.info,1,None
33,arrancar.org,1,None
34,sql-ledger.org,1,2000-09-08 16:25:09
35,backtrack-linux.org,1,2009-04-29 00:26:42


### ¿Cuáles son los dominios que podemos confirmar como sospechosos?

Considero que se podrían confirmar como sospechosos aquellos dominos que NO tienen una fecha de creación. También todos aquellos que tenían menos de un año (soy inexperto en el tema, pero consideraría que 1 año de diferencia sería ideal para determinar si es o no sospechoso) de vida. Haciendo una diferencia entre el año que se capturó el paquete (2017) y el año de creación del sitio, si la cantidad de años en la diferencia es menor o igual a 1 se podría confirmar como sospechoso

## 16. Recuerde que los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y que pueden confirmarse como dominios DGA.

In [214]:
dataframe[(dataframe["en_lista"] == 1) & (dataframe["fecha_creacion"].isna())]

,domain_tld,en_lista,fecha_creacion
3,home,1,None
18,110phpmyadmin,1,None
20,localdomain,1,None
27,ntkrnlpa.info,1,None
33,arrancar.org,1,None
38,theanime.cn,1,None
43,vpn,1,None
64,bigflickrfeed.com,1,None
66,wpad,1,None
67,lan,1,None


Dada la data mostrada anteriormente se puede confirmar que hay mínimo 8 dominios que podrían clasificarse como DGA.
Estos serían:
* ntkrnlpa.info
* 56"
* 0
* FL
* 1201
* vtlfccmfxlkgifuf.com
* 254
* ejfodfmfxlkgifuf.xyz

Digo mínimo ya que son los que tienen dos características críticas: primero que el TDL se ve que se creó de manera aleatoria y no cuentan con fecha de creación. Puede que hayan muchos más TDL que sí tienen fecha de creación y parecen creados de manera aleatoria, pero su fecha pude ser muy reciente; por lo tanto, ese TDL debería de caer en la misma clasificación de DGA debido a que su tiempo de vida es poca como para confiar en él.